In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 10000)
pd.options.display.max_columns = None

In [ ]:
df_sales_2019 = pd.read_excel('CVP_Sales_201901_201912_Raw_Data.xlsx')
df_sales_2020 = pd.read_excel('CVP_Sales_202001_202012_Raw_Data.xlsx')
df_sales_2021 = pd.read_excel('CVP_Sales_202101_202112_Raw_Data.xlsx')
df_sales_2022 = pd.read_excel('CVP_Sales_2022YTD_Raw_Data.xlsx')
df_service_2019 = pd.read_csv('CVP_Service_Raw_201901_201912.csv')
df_service_2020 = pd.read_csv('CVP_Service_Raw_202001_202012.csv')
df_service_2021 = pd.read_csv('CVP_Service_Raw_202101_202112.csv')
df_service_2022 = pd.read_csv('CVP_Service_Raw_2022YTD.csv')

In [ ]:
df_sales = pd.concat([df_sales_2019, df_sales_2020, df_sales_2021, df_sales_2022])
df_service = pd.concat([df_service_2019, df_service_2020, df_service_2021, df_service_2022])

In [ ]:
#分开赠送保养和付费保养
#如果一个人享受多次免费保养，只看最后一次
df_service['问卷回复时间'] = pd.to_datetime(df_service['问卷回复时间'])
df_service_1 = df_service.loc[df_service['是否赠送'] == '是', :]
df_service_0 = df_service.loc[(df_service['是否赠送'] == '否') & (df_service['维修保养'] == '保养'), :]
df_service_1.sort_values(by = '问卷回复时间', ascending = False, inplace=True)
df_service_1.drop_duplicates('车架号_y',keep='first',inplace=True)
df_service = pd.concat([df_service_0, df_service_1], axis = 0)

In [ ]:
#筛选填写评论的用户，过滤掉没有认真填问卷的人
df_sales = df_sales.loc[(df_sales['1.1 总体购车体验及感受(备注)'].notna()) | (df_sales['2.14 您是否愿意将福特品牌推荐给其他人？(备注)'].notna()),:]
df_service = df_service.loc[(df_service['1.1 请您评价此次4S店总体服务体验(备注)'].notna()) | (df_service['2.15 根据您的体验，您将福特品牌推荐给他人的可能性有多大？(备注)'].notna()),:]

In [ ]:
#合并
df = df_sales.merge(df_service, on = ['车架号','用户手机号'], how = 'left')

下面看一下用户在没有免费保养的情况下的保养周期，用来过滤在外面做过保养再回来的情况

In [ ]:
#先筛选去过的
df1 = df.loc[df['工单号'].notna(),:]
df1['time_diff'] = pd.to_datetime(df1['问卷回复时间_y']) - pd.to_datetime(df1['问卷回复时间_x'])
#看赠送保养最后一次的时间
maintain_free = pd.DataFrame(df1.loc[df['是否赠送'] == '是',:].groupby('车架号')['time_diff'].max())
#付费保养第一次的时间
maintain_pay = pd.DataFrame(df1.loc[df['是否赠送'] == '否',:].groupby('车架号')['time_diff'].min())
#时间间隔
df_maintain = maintain_free.merge(maintain_pay, on = '车型_x',how = 'inner')
df_maintain['time_diff'] = df_maintain['time_diff_y'] - df_maintain['time_diff_x']
#看看第一次付费和最后一次免费的时间间隔
df_maintain['time_diff'].describe()

In [ ]:
#只取240天以内的
df['time_diff'] = pd.to_datetime(df['问卷回复时间_y']) - pd.to_datetime(df['问卷回复时间_x'])
df = df.loc[df['time_diff'] <= '240days', :]

把用户的评论文本拿出来去做文本分类，把分类结果作为一个特征。销售问卷和售后问卷各有两个文本问题，我们优先用“总体体验”

In [ ]:
#文本拿出去做文本分类
df.reset_index(inplace= True, drop=True)
df['销售文本'] = df.apply(lambda df: df['2.14 您是否愿意将福特品牌推荐给其他人？(备注)'] if not df['1.1 总体购车体验及感受(备注)'] else  df['1.1 总体购车体验及感受(备注)'], axis= 1)
df['售后文本'] = df.apply(lambda df: df['2.15 根据您的体验，您将福特品牌推荐给他人的可能性有多大？(备注)'] if not df['1.1 请您评价此次4S店总体服务体验(备注)'] else  df['1.1 请您评价此次4S店总体服务体验(备注)'], axis= 1)
df[['销售文本']].to_excel('sales_text_all.xlsx',index = True)
df[['售后文本']].to_excel('service_text_all.xlsx',index = True)

In [ ]:
#做完的文本分类，加入原数据
df_sales_text = pd.read_excel('prediction_sales.xlsx')
df_service_text = pd.read_excel('prediction_service.xlsx')
df['sales_aspect'] = df_sales_text['prediction_aspect']
df['sales_sentiment'] = df_sales_text['prediction_sentiment']
df['service_aspect'] = df_service_text['prediction_aspect']
df['service_sentiment'] = df_service_text['prediction_sentiment']

In [ ]:
df = df.loc[(df['是否赠送'] == '否') & (df['维修保养'] == '保养'), :]

In [ ]:
df = df[['车架号_x','车型_x','大区_x','小区_x','问卷回复时间_x','问卷回复时间_y','1.1 总体购车体验及感受','1.2 请您评价销售顾问在入店接待环节的表现',
    '1.2 店内环境设施','1.4 请您评价销售顾问在介绍产品环节的表现',
    '1.5 请您评价支付/购买环节的体验','1.8 您选择的付款方式是什么？',
    '1.9 请您评价提新车环节的体验','2.11 购车过程便捷，省心的程度如何？',
    '2.12 您是否愿意将这家4S店推荐给其他人？','2.13 您是否信任这家4S店？',
    '2.14 您是否愿意将福特品牌推荐给其他人？','2.15 您是否信赖福特这个品牌？',
   '工单号','1.1 请您评价此次4S店总体服务体验','1.2 请您评价服务人员在接待报价环节的表现',
    '1.3 您认为哪些方面需要改善','1.4 请您评价在店等待环节的体验','1.5 您认为哪些方面需要改善',
    '1.6 请您评价结算交车环节的体验','1.7 您认为哪些方面需要改善','1.8 请您评价此次维修保养质量',
    '1.9 请问：此次进店，您的车是一次维修保养好的吗？','1.10 如您的车未一次修好，请选择原因',
    '2.11 请您评分：此次维修保养过程便捷，省心的程度','2.12 请您评分：这家4S店值得信任',
    '2.13 根据您的体验，您将4S店推荐给他人的可能性有多大？','2.14 请您评分：福特品牌值得信任',
    '2.15 根据您的体验，您将福特品牌推荐给他人的可能性有多大？','sales_aspect','sales_sentiment','service_aspect','service_sentiment']]

In [ ]:
df['target'] = df.apply(lambda x: '1' if x['工单号'].notna() else '0', axis=1)

In [ ]:
df.to_excel('df_processed.xlsx')